In [ ]:
!pip install --force-reinstall dist/alquimia_fair_forge-0.0.1.tar.gz -q

In [ ]:
from fair_forge.metrics import Humanity
from helpers.dataset import CustomRetriever

## Emotional entropy
Based on NRC emotional lexicon and Plutchik eight basic emotions

TODO:
- Implement Emotion matching
- Implement Language Style Matching, LSM
- Implement Agreeableness
- Implement Empathy, Empathic Concern

In [3]:
metrics = Humanity.run(CustomRetriever)

In [ ]:
metrics[0]

In [ ]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
humanity_index = f"{dataset}-humanity"

In [3]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
mapping_humanity = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "humanity_assistant_emotional_entropy": {"type": "float"},
      "humanity_ground_truth_spearman": {"type": "float"},
      "humanity_assistant_anger": {"type": "float"},
      "humanity_assistant_anticipation": {"type": "float"},
      "humanity_assistant_disgust": {"type": "float"},
      "humanity_assistant_fear": {"type": "float"},
      "humanity_assistant_joy": {"type": "float"},
      "humanity_assistant_sadness": {"type": "float"},
      "humanity_assistant_surprise": {"type": "float"},
      "humanity_assistant_trust": {"type": "float"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}
recreate_index(humanity_index, mapping_humanity)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": humanity_index,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")